# Politics in Northwestern

In [1]:
SCHOOL="northwestern"
SUBJECT="opinion"
START_YEAR=2009
FINAL_YEAR=2023
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(3308, 6)

In [7]:
df.head(2)

,title,url,body,year,month,day
0,Mayfest: An open letter to the Northwestern co...,https://dailynorthwestern.com/2023/04/04/opini...,Mayfest: An open letter to the Northwestern co...,2023,4,4
1,Duda: Illinois politicians are choosing megaco...,https://dailynorthwestern.com/2023/04/03/opini...,Duda: Illinois politicians are choosing megaco...,2023,4,3


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-0.448492


In [11]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009], dtype=int64)

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2010, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2010 ... 34
2010	34	104.14887699999997	3.0632022647058816
Year: 2011 ... 163
2011	163	244.18337999999986	1.4980575460122691
Year: 2012 ... 296
2012	296	614.8792189999999	2.0772946587837837
Year: 2013 ... 349
2013	349	419.9189260000001	1.2032060916905447
Year: 2014 ... 388
2014	388	95.76497399999998	0.24681694329896903
Year: 2015 ... 368
2015	368	-769.4473370000005	-2.090889502717393
Year: 2016 ... 327
2016	327	-1281.763985999999	-3.9197675412844006
Year: 2017 ... 303
2017	303	-1686.4253100000012	-5.565760099009905
Year: 2018 ... 266
2018	266	-1399.1474299999988	-5.2599527443608975
Year: 2019 ... 247
2019	247	-1175.1517359999998	-4.757699336032388
Year: 2020 ... 179
2020	179	-859.3722629999997	-4.80096236312849
Year: 2021 ... 144
2021	144	-751.820147	-5.220973243055556
Year: 2022 ... 157
2022	157	-478.32951500000013	-3.046684808917198


In [13]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)